# Translation Demo

## Description

This notebook provides an interactive demo for text translation using pre-trained Llama models from Hugging Face's `transformers` library, integrated with `gradio` for a user-friendly interface. Users can select from multiple languages to translate text from English into the desired target language.

### Main Features:
- Select from a variety of pre-trained Llama models for text translation.
- Input text in English and translate it into one of 20 supported target languages, including French, Spanish, German, Chinese, and more.
- Display the translated text in a dedicated output textbox.
- Cache models to avoid reloading and improve performance.
- Select the appropriate translation pipeline based on the chosen target language.


## Log in to Hugging Face Hub

In this cell, we import the `login` function from the Hugging Face Hub and call it to authenticate with your Hugging Face account. This step is required to access private models, datasets, and other resources hosted on Hugging Face.

### Directions to Generate Access Token:
1. Go to the [Hugging Face website](https://huggingface.co/).
2. Log in to your Hugging Face account.
3. Navigate to your **profile icon** on the top right, and click **Settings**.
4. Under **Access Tokens** (on the left sidebar), click **New Token** to generate a new access token.
5. Copy the generated token.

### Usage:
When you run this cell, you'll be prompted to paste the access token, which grants access to your Hugging Face resources.

> **Do not share your Access Tokens with anyone**

In [ ]:
from huggingface_hub import login
login()

## Import required libraries

In [ ]:
import gradio as gr
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

In [ ]:
device = 0 if torch.cuda.is_available() else -1

## Basic Usage

Below there is a list of available Llama models to choose from. The function, `load_translation_model`, loads the selected Llama model along with its tokenizer. It uses Hugging Face's `AutoModelForSeq2SeqLM` and `AutoTokenizer` to load the pre-trained model and then returns a translation pipeline. The user can select the target language for translation from a dropdown menu and translate the provided text into languages such as French, Spanish, German, and more.


In [ ]:
llama_models = {
    "Llama 3 70B Instruct": "meta-llama/Meta-Llama-3-70B-Instruct",
    "Llama 3 8B Instruct": "meta-llama/Meta-Llama-3-8B-Instruct",
    "Llama 3.1 70B Instruct": "meta-llama/Llama-3.1-70B-Instruct",
    "Llama 3.1 8B Instruct": "meta-llama/Llama-3.1-8B-Instruct",
    "Llama 3.2 3B Instruct": "meta-llama/Llama-3.2-3B-Instruct",
    "Llama 3.2 1B Instruct": "meta-llama/Llama-3.2-1B-Instruct",
}

In [ ]:
def load_translation_model(model_name):
    """Load the specified Llama translation model."""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    translator = pipeline('translation_en_to_fr', model=model, tokenizer=tokenizer, device=device)
    return translator

Cache models to avoid reloading

In [ ]:
model_cache = {}

In [ ]:
def translate_text(text, model_choice, target_language):
    """Translate text using the selected Llama model and target language."""
    if model_choice not in model_cache:
        model_cache[model_choice] = load_translation_model(llama_models[model_choice])
    translator = model_cache[model_choice]
    
    # Map target language to the appropriate translation task
    language_map = {
        "French": "translation_en_to_fr",
        "Spanish": "translation_en_to_es",
        "German": "translation_en_to_de",
        "Italian": "translation_en_to_it",
        "Portuguese": "translation_en_to_pt",
        "Dutch": "translation_en_to_nl",
        "Russian": "translation_en_to_ru",
        "Chinese": "translation_en_to_zh",
        "Japanese": "translation_en_to_ja",
        "Korean": "translation_en_to_ko",
        "Arabic": "translation_en_to_ar",
        "Hindi": "translation_en_to_hi",
        "Bengali": "translation_en_to_bn",
        "Greek": "translation_en_to_el",
        "Turkish": "translation_en_to_tr",
        "Swedish": "translation_en_to_sv",
        "Norwegian": "translation_en_to_no",
        "Danish": "translation_en_to_da",
        "Finnish": "translation_en_to_fi",
        "Polish": "translation_en_to_pl"
    }
    
    translation_task = language_map.get(target_language, "translation_en_to_fr")
    translator = pipeline(translation_task, model=translator.model, tokenizer=translator.tokenizer, device=device)
    
    result = translator(text)[0]
    return result['translation_text']

The cell below defines a simple Gradio interface for text translation. It includes a dropdown menu where users can select a specific Llama model for translation, a textbox where users can input the English text to be translated, and another dropdown to choose the target language from a list of available languages. 


In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("<h1><center>Translation with Llama Models</center></h1>")
    model_choice = gr.Dropdown(list(llama_models.keys()), label="Select Llama Model")
    input_text = gr.Textbox(label="Enter text to translate", lines=4)
    target_language = gr.Dropdown(
        ["French", "Spanish", "German", "Italian", "Portuguese", "Dutch", "Russian", "Chinese", "Japanese", "Korean", "Arabic", "Hindi", "Bengali", "Greek", "Turkish", "Swedish", "Norwegian", "Danish", "Finnish", "Polish"],
        label="Select Target Language"
    )
    output_text = gr.Textbox(label="Translated Text")
    gr.Button("Translate").click(translate_text, [input_text, model_choice, target_language], output_text)

Launch the interface

In [ ]:
demo.launch()